## Sixth Session (Related to the Course Project)


## Graph Regression with [Deep Graph Library (DGL)](https://docs.dgl.ai/index.html) for the graduate course "[Graph Machine learning](https://github.com/zahta/graph_ml)"

### Dataset: Lipophilicity

##### by [Zahra Taheri](https://github.com/zahta), 06 June 2023



### This Tutorial Is Prepared Based on the Following References

- [FunQG: Molecular Representation Learning via Quotient Graphs](https://pubs.acs.org/doi/10.1021/acs.jcim.3c00445)
- [Supporting Information of FunQG](https://pubs.acs.org/doi/suppl/10.1021/acs.jcim.3c00445/suppl_file/ci3c00445_si_001.pdf)
- [GitHub Repository of FunQG](https://github.com/hhaji/funqg)

In [2]:
pip install  dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.7 MB/s eta 0:00:00


In [3]:
pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 66.1 MB/s eta 0:00:00
  Created whee

In [4]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv
from dgl.nn import GINConv
from dgl.nn import SAGEConv
from dgl.nn import GATConv
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Set Path

In [6]:
current_dir = "/content/drive/MyDrive/graph_data20.zip"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir
shutil.unpack_archive(path_save, folder_data_temp)

#### Custom PyTorch Datasets

In [7]:
""" Regression Dataset """
class DGLDatasetReg(torch.utils.data.Dataset):
    def __init__(self, address, transform=None, train=False, scaler=None, scaler_regression=None):
        # Indicates whether the dataset is for training or not
        self.train = train
        # Scaler for preprocessing the labels
        self.scaler = scaler
      
        # Load the graph dataset from the specified address
        self.data_set, train_labels_masks_globals = dgl.load_graphs(address+".bin")
        num_graphs = len(self.data_set)
      
        # Extract labels, masks, and globals from the loaded dataset
        self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
        self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
        self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
      
        # Transformation function for data augmentation (if any)
        self.transform = transform
     
        # Scaler specific to regression tasks
        self.scaler_regression = scaler_regression

    def scaler_method(self):
        # Fit the scaler to the labels if the dataset is for training
        if self.train:
            scaler = preprocessing.StandardScaler().fit(self.labels)
            self.scaler = scaler
        return self.scaler

    def __len__(self):
        # Return the total number of graphs in the dataset
        return len(self.data_set)

    def __getitem__(self, idx):
        if self.scaler_regression:
            """ With Scaler """
            # Return the graph, transformed and scaled labels, masks, and globals
            return self.data_set[idx], torch.tensor(self.scaler.transform(self.labels)[idx]).float(), self.masks[idx], self.globals[idx]
        else:
            """ Without Scaler """
            # Return the graph, original labels, masks, and globals
            return self.data_set[idx], self.labels[idx].float(), self.masks[idx], self.globals[idx]
```

Now the code has English comments to explain the different parts of the code. I apologize for the oversight in my previous response.

#### Defining Train, Validation, and Test Set

In [9]:
# Initialize the scaler dictionary
scaler = {}

# Create the path for temporary data with the specified scaffold index
path_data_temp = folder_data_temp + "scaffold" + "_" + str(0)

# Create the training dataset object with the specified address and set it for training
train_set = DGLDatasetReg(address=path_data_temp + "_train", train=True)

# Use the scaler method of the training dataset to obtain the scaler object
scaler = train_set.scaler_method()

# Create the validation dataset object with the specified address and the obtained scaler
val_set = DGLDatasetReg(address=path_data_temp + "_val", scaler=scaler)

# Create the test dataset object with the specified address and the obtained scaler
test_set = DGLDatasetReg(address=path_data_temp + "_test", scaler=scaler)

# Print the lengths of the training, validation, and test sets
print(len(train_set), len(val_set), len(test_set))



3360 420 420


#### Data Loader

In [10]:
def collate(batch):
    
    
    graphs = [e[0] for e in batch]
    
    g = dgl.batch(graphs)

    

    
    labels = [e[1] for e in batch]
   
    labels = torch.stack(labels, 0)

    
   
    masks = [e[2] for e in batch]
    
    masks = torch.stack(masks, 0)

    

    
    globals = [e[3] for e in batch]
   
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [11]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

#### Defining A GNN

##### Some Variables

In [12]:
#Bace dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


# GCN 2 Layer

In [12]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [20]:
import math
from sklearn.metrics import mean_squared_error

def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
    # Set the model to evaluation mode
    model.eval()
  
    # Define the loss function using mean squared error with sum reduction
    loss_sum = nn.MSELoss(reduction='sum')  # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  
    # Initialize the final loss variable
    final_loss = 0
  
    # Save the current state of the random number generator
    state = torch.get_rng_state()
  
    # Disable gradient computation during inference
    with torch.no_grad():
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            # Perform forward pass to obtain predictions from the model
            prediction = model(mol_dgl_graph, globals)
          
            # Inverse transform the predictions using the provided scaler
            prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
          
            # Inverse transform the labels using the provided scaler
            labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
          
            # Calculate the loss between the predictions and labels
            loss = loss_sum(prediction, labels)
          
            # Accumulate the loss
            final_loss += loss.item()
      
        # Calculate the final loss by dividing by the validation size and taking the square root
        final_loss /= val_size
        final_loss = math.sqrt(final_loss)
  
    # Return the normalized final loss
    return final_loss / num_tasks


In [14]:
def loss_func(output, label, mask, num_tasks):
    # Create a tensor of ones with shape (1, num_tasks) as the positive weight
    pos_weight = torch.ones((1, num_tasks))
  
    # Initialize the criterion as the mean squared error loss with no reduction
    criterion = nn.MSELoss(reduction='none')
  
    # Calculate the element-wise loss by multiplying the mask with the criterion output
    loss = mask * criterion(output, label)
  
    # Calculate the sum of the loss and divide it by the sum of the mask to obtain the average loss
    loss = loss.sum() / mask.sum()
  
    # Return the calculated loss
    return loss


In [15]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [26]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [27]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [28]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 5.456 | Valid Score: 2.392
 
Epoch: 1/100 | Best Valid Score Until Now: 2.392 

Save checkpoint
Epoch: 2/100 | Training Loss: 3.242 | Valid Score: 1.799
 
Epoch: 2/100 | Best Valid Score Until Now: 1.799 

Save checkpoint
Epoch: 3/100 | Training Loss: 2.028 | Valid Score: 1.540
 
Epoch: 3/100 | Best Valid Score Until Now: 1.540 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.784 | Valid Score: 1.515
 
Epoch: 4/100 | Best Valid Score Until Now: 1.515 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.748 | Valid Score: 1.500
 
Epoch: 5/100 | Best Valid Score Until Now: 1.500 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.704 | Valid Score: 1.487
 
Epoch: 6/100 | Best Valid Score Until Now: 1.487 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.671 | Valid Score: 1.474
 
Epoch: 7/100 | Best Valid Score Until Now: 1.474 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.650 | Valid Score: 1.462
 
Epoch: 8/100 | Best Valid Score Until Now: 

# GCN 3 Layer

In [29]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")


In [30]:
def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
       prediction = model(mol_dgl_graph, globals)
       prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
       labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
       loss = loss_sum(prediction, labels)
       final_loss += loss.item()

    final_loss /= val_size
    final_loss = math.sqrt(final_loss)

  return final_loss / num_tasks

In [31]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [32]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [33]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [34]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [35]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 6.726 | Valid Score: 2.726
 
Epoch: 1/100 | Best Valid Score Until Now: 2.726 

Save checkpoint
Epoch: 2/100 | Training Loss: 4.331 | Valid Score: 2.085
 
Epoch: 2/100 | Best Valid Score Until Now: 2.085 

Save checkpoint
Epoch: 3/100 | Training Loss: 2.483 | Valid Score: 1.598
 
Epoch: 3/100 | Best Valid Score Until Now: 1.598 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.808 | Valid Score: 1.514
 
Epoch: 4/100 | Best Valid Score Until Now: 1.514 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.747 | Valid Score: 1.501
 
Epoch: 5/100 | Best Valid Score Until Now: 1.501 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.701 | Valid Score: 1.487
 
Epoch: 6/100 | Best Valid Score Until Now: 1.487 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.679 | Valid Score: 1.474
 
Epoch: 7/100 | Best Valid Score Until Now: 1.474 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.637 | Valid Score: 1.462
 
Epoch: 8/100 | Best Valid Score Until Now: 

# GraphSAGE 2 Layer

In [27]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size,aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks, aggregator_type='mean')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [28]:
def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
       prediction = model(mol_dgl_graph, globals)
       prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
       labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
       loss = loss_sum(prediction, labels)
       final_loss += loss.item()

    final_loss /= val_size
    final_loss = math.sqrt(final_loss)

  return final_loss / num_tasks

In [29]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [30]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [31]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [32]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [33]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 4.683 | Valid Score: 2.003
 
Epoch: 1/100 | Best Valid Score Until Now: 2.003 

Save checkpoint
Epoch: 2/100 | Training Loss: 2.308 | Valid Score: 1.608
 
Epoch: 2/100 | Best Valid Score Until Now: 1.608 

Save checkpoint
Epoch: 3/100 | Training Loss: 1.851 | Valid Score: 1.543
 
Epoch: 3/100 | Best Valid Score Until Now: 1.543 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.761 | Valid Score: 1.520
 
Epoch: 4/100 | Best Valid Score Until Now: 1.520 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.698 | Valid Score: 1.499
 
Epoch: 5/100 | Best Valid Score Until Now: 1.499 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.656 | Valid Score: 1.482
 
Epoch: 6/100 | Best Valid Score Until Now: 1.482 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.609 | Valid Score: 1.466
 
Epoch: 7/100 | Best Valid Score Until Now: 1.466 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.578 | Valid Score: 1.453
 
Epoch: 8/100 | Best Valid Score Until Now: 

# GraphSAGE 3 Layer

In [43]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size,aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size,aggregator_type='mean')
        self.conv3 = SAGEConv(self.hidden_size, self.num_tasks,aggregator_type='mean')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")


In [44]:
def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
       prediction = model(mol_dgl_graph, globals)
       prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
       labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
       loss = loss_sum(prediction, labels)
       final_loss += loss.item()

    final_loss /= val_size
    final_loss = math.sqrt(final_loss)

  return final_loss / num_tasks

In [45]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [46]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [47]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [48]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [49]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 2.657 | Valid Score: 1.510
 
Epoch: 1/100 | Best Valid Score Until Now: 1.510 

Save checkpoint
Epoch: 2/100 | Training Loss: 1.665 | Valid Score: 1.429
 
Epoch: 2/100 | Best Valid Score Until Now: 1.429 

Save checkpoint
Epoch: 3/100 | Training Loss: 1.514 | Valid Score: 1.399
 
Epoch: 3/100 | Best Valid Score Until Now: 1.399 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.450 | Valid Score: 1.385
 
Epoch: 4/100 | Best Valid Score Until Now: 1.385 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.401 | Valid Score: 1.366
 
Epoch: 5/100 | Best Valid Score Until Now: 1.366 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.361 | Valid Score: 1.348
 
Epoch: 6/100 | Best Valid Score Until Now: 1.348 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.316 | Valid Score: 1.339
 
Epoch: 7/100 | Best Valid Score Until Now: 1.339 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.280 | Valid Score: 1.328
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GIN 2Layer

In [34]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GINConv(nn.Linear(self.node_feature_size, self.hidden_size), aggregator_type='sum')
        self.conv2 = GINConv(nn.Linear(self.hidden_size, self.num_tasks), aggregator_type='sum')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [35]:
def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
       prediction = model(mol_dgl_graph, globals)
       prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
       labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
       loss = loss_sum(prediction, labels)
       final_loss += loss.item()

    final_loss /= val_size
    final_loss = math.sqrt(final_loss)

  return final_loss / num_tasks

In [36]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [37]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [38]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [39]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [40]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 5.570 | Valid Score: 2.183
 
Epoch: 1/100 | Best Valid Score Until Now: 2.183 

Save checkpoint
Epoch: 2/100 | Training Loss: 2.501 | Valid Score: 1.626
 
Epoch: 2/100 | Best Valid Score Until Now: 1.626 

Save checkpoint
Epoch: 3/100 | Training Loss: 1.754 | Valid Score: 1.524
 
Epoch: 3/100 | Best Valid Score Until Now: 1.524 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.680 | Valid Score: 1.513
 
Epoch: 4/100 | Best Valid Score Until Now: 1.513 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.661 | Valid Score: 1.506
 
Epoch: 5/100 | Best Valid Score Until Now: 1.506 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.645 | Valid Score: 1.499
 
Epoch: 6/100 | Best Valid Score Until Now: 1.499 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.625 | Valid Score: 1.492
 
Epoch: 7/100 | Best Valid Score Until Now: 1.492 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.617 | Valid Score: 1.485
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GIN 3Layer

In [41]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GINConv(nn.Linear(self.node_feature_size, self.hidden_size), aggregator_type='sum')
        self.conv2 = GINConv(nn.Linear(self.hidden_size, self.hidden_size), aggregator_type='sum')
        self.conv3 = GINConv(nn.Linear(self.hidden_size, self.num_tasks), aggregator_type='sum')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")


In [42]:
def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
       prediction = model(mol_dgl_graph, globals)
       prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
       labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
       loss = loss_sum(prediction, labels)
       final_loss += loss.item()

    final_loss /= val_size
    final_loss = math.sqrt(final_loss)

  return final_loss / num_tasks

In [43]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [44]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [45]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [46]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [47]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 4.092 | Valid Score: 1.552
 
Epoch: 1/100 | Best Valid Score Until Now: 1.552 

Save checkpoint
Epoch: 2/100 | Training Loss: 1.660 | Valid Score: 1.526
 
Epoch: 2/100 | Best Valid Score Until Now: 1.526 

Save checkpoint
Epoch: 3/100 | Training Loss: 1.611 | Valid Score: 1.509
 
Epoch: 3/100 | Best Valid Score Until Now: 1.509 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.594 | Valid Score: 1.498
 
Epoch: 4/100 | Best Valid Score Until Now: 1.498 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.558 | Valid Score: 1.489
 
Epoch: 5/100 | Best Valid Score Until Now: 1.489 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.535 | Valid Score: 1.478
 
Epoch: 6/100 | Best Valid Score Until Now: 1.478 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.529 | Valid Score: 1.466
 
Epoch: 7/100 | Best Valid Score Until Now: 1.466 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.503 | Valid Score: 1.458
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GAT 2Layer

In [78]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.num_heads = self.config.get('num_heads', 1)
        self.dropout = self.config.get('dropout', 0.0)

        self.conv1 = GATConv(
            self.node_feature_size,
            self.hidden_size,
            num_heads=self.num_heads,
            feat_drop=self.dropout,
            attn_drop=self.dropout,allow_zero_in_degree=True)

        self.fc = nn.Linear(
            self.hidden_size * self.num_heads,
            self.hidden_size)
        self.conv2 = GATConv(
            self.hidden_size,
            self.num_tasks,
            num_heads=1,
            feat_drop=self.dropout,
            attn_drop=self.dropout,allow_zero_in_degree=True
       )
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")


In [79]:
def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
       prediction = model(mol_dgl_graph, globals)
       prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
       labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
       loss = loss_sum(prediction, labels)
       final_loss += loss.item()

    final_loss /= val_size
    final_loss = math.sqrt(final_loss)

  return final_loss / num_tasks

In [80]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [81]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [82]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [83]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [84]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 4.017 | Valid Score: 1.698
 
Epoch: 1/100 | Best Valid Score Until Now: 1.698 

Save checkpoint
Epoch: 2/100 | Training Loss: 1.838 | Valid Score: 1.511
 
Epoch: 2/100 | Best Valid Score Until Now: 1.511 

Save checkpoint
Epoch: 3/100 | Training Loss: 1.718 | Valid Score: 1.495
 
Epoch: 3/100 | Best Valid Score Until Now: 1.495 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.686 | Valid Score: 1.481
 
Epoch: 4/100 | Best Valid Score Until Now: 1.481 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.653 | Valid Score: 1.468
 
Epoch: 5/100 | Best Valid Score Until Now: 1.468 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.619 | Valid Score: 1.456
 
Epoch: 6/100 | Best Valid Score Until Now: 1.456 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.590 | Valid Score: 1.442
 
Epoch: 7/100 | Best Valid Score Until Now: 1.442 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.560 | Valid Score: 1.430
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GAT 3Layer

In [85]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)


        
        self.num_heads = self.config.get('num_heads', 1)

        
        self.dropout = self.config.get('dropout', 0.0)

        # GAT layer1
        self.conv1 = GATConv(self.node_feature_size,self.hidden_size,num_heads=self.num_heads,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

        # Linear layer
        self.fc = nn.Linear(self.hidden_size * self.num_heads,self.hidden_size)

        # GAT layer2
        self.conv2 = GATConv(self.hidden_size,self.hidden_size,num_heads=1,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

        # GAT layer3
        self.conv3 = GATConv(self.hidden_size,self.num_tasks,num_heads=1,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Third GAT layer
        if self.num_tasks == 1:
            h = self.conv3(mol_dgl_graph, h).squeeze(1)
        else:
            hs = []
            for i in range(self.num_tasks):
                hi = self.conv3(mol_dgl_graph, h).squeeze(1)
                hs.append(hi)
            h = torch.stack(hs, dim=1)

        mol_dgl_graph.ndata["h"] = h

        if self.num_tasks == 1:
            return dgl.mean_nodes(mol_dgl_graph, "h")
        else:
            return dgl.mean_nodes(mol_dgl_graph, "h"), h


In [86]:
def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
       prediction = model(mol_dgl_graph, globals)
       prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
       labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
       loss = loss_sum(prediction, labels)
       final_loss += loss.item()

    final_loss /= val_size
    final_loss = math.sqrt(final_loss)

  return final_loss / num_tasks

In [87]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [88]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [89]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [90]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [91]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 2.427 | Valid Score: 1.567
 
Epoch: 1/100 | Best Valid Score Until Now: 1.567 

Save checkpoint
Epoch: 2/100 | Training Loss: 1.802 | Valid Score: 1.489
 
Epoch: 2/100 | Best Valid Score Until Now: 1.489 

Save checkpoint
Epoch: 3/100 | Training Loss: 1.632 | Valid Score: 1.443
 
Epoch: 3/100 | Best Valid Score Until Now: 1.443 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.549 | Valid Score: 1.421
 
Epoch: 4/100 | Best Valid Score Until Now: 1.421 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.505 | Valid Score: 1.413
 
Epoch: 5/100 | Best Valid Score Until Now: 1.413 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.485 | Valid Score: 1.401
 
Epoch: 6/100 | Best Valid Score Until Now: 1.401 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.448 | Valid Score: 1.398
 
Epoch: 7/100 | Best Valid Score Until Now: 1.398 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.436 | Valid Score: 1.391
 
Epoch: 8/100 | Best Valid Score Until Now: 